## Query
The most basic search that can be performed is the query.  The query is a free-formed text expression, similar to the type of query you can perform within Google, matching against documents within the specified view.

#### NuGet Packages

In [1]:
#r "nuget:Refinitiv.DataPlatform.Content, 1.0.0-alpha3"
#r "nuget:Microsoft.Data.Analysis"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Refinitiv.DataPlatform.Content version 1.0.0-alpha3

Installed package Microsoft.Data.Analysis version 0.4.0

In [2]:
using Newtonsoft.Json.Linq;
using Refinitiv.DataPlatform.Content.SearchService;
using Refinitiv.DataPlatform.Core;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Linq;
using System;

#### Table Output
Helper routine to output data in a table format.

In [3]:
Formatter.Register<IList<JObject>>((hits, writer) =>
{
    if (hits.Count > 0)
    {
        var fields = new List<String>();
        var rows = new List<ICollection<IHtmlContent>>();
        var data = new Dictionary<string, IHtmlContent>();

        foreach(var hit in hits)
        {
            var cells = new List<IHtmlContent>();
            foreach (var val in hit.Properties())
            {
                if ( !fields.Any(item => item.Equals(val.Name)) )
                    fields.Add(val.Name);
                data[val.Name] = td(val.Value.ToString());
            }
            rows.Add(new List<IHtmlContent>(data.Values));
            data.Keys.ToList().ForEach(x => data[x] = td(""));
        }
        
        var headers = new List<IHtmlContent>();
        headers.AddRange(fields.Select(c => (IHtmlContent)th(c)));

        var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
        writer.Write(t);
    }
}, "text/html");

In [4]:
// Create a session into the desktop
var session = CoreFactory.CreateSession(new DesktopSession.Params()
                            .AppKey("Your API Key here")
                            .OnState((s, state, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (State: {state})"))
                            .OnEvent((s, eventCode, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (Event: {eventCode})")));
session.Open();

17/12/2020 12:32:51 PM:Session is Pending. (State: Pending)
17/12/2020 12:32:51 PM:{
  "Contents": "Desktop Session Successfully Authenticated"
}. (Event: SessionAuthenticationSuccess)
17/12/2020 12:32:51 PM:Session is Opened. (State: Opened)


#### Query - Default behavior

In [5]:
// The default syntax for search is to specify a query expression.  By default, search will return a default set of 
// fields (Properties).
var response = Search.Definition(Search.View.SearchAll).Query("IBM")
                                                       .GetData();
response.Data.Hits

BusinessEntity DocumentTitle PI PermID RIC ORGANISATION International Business Machines Corp, Public Company 37036 QUOTExEQUITY International Business Machines Corp, Ordinary Share, NYSE Consolidated 1097326 55839165994 IBM ORGANISATION IBM Credit LLC, Private Company 99244 ORGS IBM Retirement Fund Trust, Private Company 4297694029 ORGANISATION Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4, Private Company 18062670 ORGANISATION SG Stuttgart Vaihingen BM-Campus 1 UG haftungsbeschraenkt, Private Company 27968389 QUOTExEQUITY Eurex International Business Machines Equity Future Chain Contract , Equity Future, USD, Eurex 48924732 21481052421 0#IBMF: QUOTExEQUITY Euronext Amsterdam IBM Dividend Future Chain Contracts, Equity Future, USD, Euronext Amsterdam 259118763 21612423771 0#IBMDF: QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 1, Equity Future, USD, Eurex 49450681 21481052892 IBMFc1 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 2, Equity Future, USD, Eurex 50092347 21481053949 IBMFc2

In [6]:
// By default, 10 documents will be returned.  Let's override that.
response = Search.Definition(Search.View.SearchAll).Query("IBM")
                                                   .Top(20)
                                                   .GetData();
response.Data.Hits

BusinessEntity DocumentTitle PI PermID RIC ORGANISATION International Business Machines Corp, Public Company 37036 QUOTExEQUITY International Business Machines Corp, Ordinary Share, NYSE Consolidated 1097326 55839165994 IBM ORGANISATION IBM Credit LLC, Private Company 99244 ORGS IBM Retirement Fund Trust, Private Company 4297694029 ORGANISATION Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4, Private Company 18062670 ORGANISATION SG Stuttgart Vaihingen BM-Campus 1 UG haftungsbeschraenkt, Private Company 27968389 QUOTExEQUITY Eurex International Business Machines Equity Future Chain Contract , Equity Future, USD, Eurex 48924732 21481052421 0#IBMF: QUOTExEQUITY Euronext Amsterdam IBM Dividend Future Chain Contracts, Equity Future, USD, Euronext Amsterdam 259118763 21612423771 0#IBMDF: QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 1, Equity Future, USD, Eurex 49450681 21481052892 IBMFc1 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 2, Equity Future, USD, Eurex 50092347 21481053949 IBMFc2 QUOTExEQUITY Euronext Amsterdam IBM Single Stock Dividend Future Continuation 1, Equity Future, USD, Euronext Amsterdam 260213021 21613372305 IBMDFc1 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 3, Equity Future, USD, Eurex 50092348 21481053950 IBMFc3 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 4, Equity Future, USD, Eurex 50092349 21481053951 IBMFc4 QUOTExEQUITY Euronext Amsterdam IBM Single Stock Dividend Future Continuation 2, Equity Future, USD, Euronext Amsterdam 260213022 21613372306 IBMDFc2 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 5, Equity Future, USD, Eurex 50092350 21481053952 IBMFc5 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 6, Equity Future, USD, Eurex 50092351 21481053953 IBMFc6 QUOTExEQUITY Euronext Amsterdam IBM Single Stock Dividend Future Continuation 3, Equity Future, USD, Euronext Amsterdam 260213023 21613372307 IBMDFc3 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 7, Equity Future, USD, Eurex 50092352 21481053954 IBMFc7 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 8, Equity Future, USD, Eurex 50092353 21481053955 IBMFc8 QUOTExEQUITY Euronext Amsterdam IBM Single Stock Dividend Future Continuation 4, Equity Future, USD, Euronext Amsterdam 260213024 21613372308 IBMDFc4

#### Query expressions
If we were to type in the following expression, we might come away believing the query will list all personnel within IBM that have a title
of CEO.  While, the results will indicate people related to IBM and people with a title of CEO, we cannot assume they are both.  That is, 
we will see results with the term IBM and CEO listed in the document.

In [7]:
response = Search.Definition(Search.View.SearchAll).Query("IBM CEO")
                                                   .GetData();
response.Data.Hits

PERSON,"Arvind Krishna - International Business Machines Corp - Chief Executive Officer, Director",34417810693,34417810693
PERSON,"Rudy S. Karsan - Kenexa Corp - Chairman of the Board, Chief Executive Officer",34413248256,34413248256
PERSON,"Mohammed Farooq - Gravitant Inc - Chairman of the Board, Chief Executive Officer, Co-Founder",34415058863,34415058863
PERSON,"Deidre Paknad - Pss Systems Inc - President, CEO, Board Member",34414057544,34414057544
PERSON,Ron S Dembo - Algorithmics Inc - President & CEO,34414091235,34414091235
PERSON,Frank Chisholm - Aptsoft Corp - CEO & Founder,34414058841,34414058841
PERSON,Michael Knagenhjelm - Metamerge As(Ibm Corp) - Chief Executive Officer,34414139687,34414139687
PERSON,Bruce Miller - Tarian Software(Ibm) - Chief Executive Officer,34414399995,34414399995


What might give a more favorable result would be to query all documents related to people that contain the term '*IBM*'. From the result, I can visually see
titles and optionally filter out people that are not CEO's.  I can do this by selecting the '**People**' view.
In addition, I'm going to choose specific *child* elements of the **DocumentTitle** field.  As we see in our output from the above requests, the 
'**DocumentTitle**' is selected by default.  This property is a convenient collection of elements that nicely describes the document returned and is
made up of multiple sub-properties such as: '**DTSubjectName**', '**DTSimpleType**', '**DTSource**', and '**DTCharacteristics**'. 

By doing this, I can now see a nice
break-down of the titles (DTCharacteristics), name (DTSubjectName) and company (DTSimpleType).

In [8]:
var select = "DocumentTitle, DTSubjectName, DTSimpleType, DTSource, DTCharacteristics";

response = Search.Definition(Search.View.People).Query("IBM")
                                                .Select(select)
                                                .GetData();
response.Data.Hits

Ginni M. Rometty - International Business Machines Corp - Executive Chairman of the Board,Ginni M. Rometty,International Business Machines Corp,Executive Chairman of the Board
Jim M. Whitehurst - International Business Machines Corp - President,Jim M. Whitehurst,International Business Machines Corp,President
"Arvind Krishna - International Business Machines Corp - Chief Executive Officer, Director",Arvind Krishna,International Business Machines Corp,"Chief Executive Officer, Director"
"James J. Kavanaugh - International Business Machines Corp - Senior Vice President, Chief Financial Officer, Finance and operation",James J. Kavanaugh,International Business Machines Corp,"Senior Vice President, Chief Financial Officer, Finance and operation"
"Diane J. Gherson - International Business Machines Corp - Chief Human Resource Officer, Senior Vice President",Diane J. Gherson,International Business Machines Corp,"Chief Human Resource Officer, Senior Vice President"
John E. Kelly - International Business Machines Corp - Executive Vice President,John E. Kelly,International Business Machines Corp,Executive Vice President
Kenneth M. Keverian - International Business Machines Corp - Senior Vice President - Corporate Strategy,Kenneth M. Keverian,International Business Machines Corp,Senior Vice President - Corporate Strategy
Michelle H. Browdy - International Business Machines Corp - Senior Vice President - Legal and Regulatory Affairs and General Counsel,Michelle H. Browdy,International Business Machines Corp,Senior Vice President - Legal and Regulatory Affairs and General Counsel
"Robert F. Del Bene - International Business Machines Corp - Vice President, Controller",Robert F. Del Bene,International Business Machines Corp,"Vice President, Controller"
Mike L. Eskew - International Business Machines Corp - Lead Independent Director,Mike L. Eskew,International Business Machines Corp,Lead Independent Director


In [9]:
// Search for a specific ISIN using query
response = Search.Definition(Search.View.SearchAll).Query("US046353AQ14")
                                                   .GetData();
response.Data.Hits

INSTRUMENTxFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",46645702869,0x00102cb50fde0d55,046353AQ1=
QUOTExFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, MARKETAXESS L2",,0x00102c9a7ade0d0f,046353AQ1=2M
QUOTExFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, CREDIT SUISSE TRADEWEB PRICE",,0x00102cbda5b71586,US046353AQ14=CSTW
QUOTExFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, MARKETAXESS COMPOSITE PLUS",,0x00102ce0bcbf19c6,US046353AQ14=MKCP
QUOTExFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, TDS TRADEWEB PRICE",,0x00102c435ce30dcd,US046353AQ14=TDTW
QUOTExFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, TRADEWEB",,0x00102c4f5ce30dc1,US046353AQ14=TE
QUOTExFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, TRADEWEB",,0x00102c1968121c43,046353AQ1=TWEB
QUOTExFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, GFI GROUP",,0x00102cadc88d0eda,US046353AQ14=GFGL
QUOTExFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, MARKETAXESS AXESSALL",,0x00102cc599441a3e,US046353AQ14=MKAX
QUOTExFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, TWEB EUCR MARKS",,0x00102c60801b1bdc,US046353AQ14=TEMK


In [10]:
// Narrow the search to only instruments
response = Search.Definition(Search.View.Instruments).Query("US046353AQ14")
                                                     .GetData();
response.Data.Hits

INSTRUMENTxFIXEDINCOMExGOVCORP,"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",46645702869,0x00102cb50fde0d55,046353AQ1=


#### Query Syntax
A query phrase can support some basic syntax including wildcards, boolean expressions and exact phrase matching.

In [11]:
// Use a specifc term in quotes and the boolean operator to locate specific tankers - use the Content layer 
// to print additional details.
response = Search.Definition(Search.View.SearchAll).Query("'Caribbean Tanker' and Baltic")
                                                   .GetData();
Console.WriteLine($"Matched a total of {response.Data.Total} documents");
response.Data.Hits

Matched a total of 14 documents


BusinessEntity DocumentTitle PI RIC PermID QUOTExCOMMODITY BALTIC DIRTY TANKER TD9 70,000mt, Caribbean to US Gulf, Commodity Index, The Baltic Exchange 621981 .BACG QUOTExCOMMODITY Baltic Exchange Dirty Tanker TD21 Caribbean to US Gulf (50,000mt fuel oil), Commodity Spot, The Baltic Exchange 175653765 .BACR 21717867959 QUOTExCOMMODITY Baltic Exchange Dirty Tanker TD9 TCE�Caribbean to US Gulf Time Charter, Commodity Spot, The Baltic Exchange 48950908 .BAAFRTD9 21717782063 QUOTExCOMMODITY Baltic Exchange Dirty Tanker TD21 TCE Caribbean to US Gulf (50,000mt fuel oil) Time Charter, Commodity Spot, The Baltic Exchange 325658314 .BAAFRTD21 21717782060 PHYSICALASSETxVESSEL ARDMORE ENDURANCE, Gasoline; Medium, ARDMORE SHIPPING BERMUDA LTD, North Atlantic Ocean 77309531000 C}KM7309531000 77309531000 PHYSICALASSETxVESSEL CARONI PLAIN, Medium, METHANOL HOLDINGS, North Atlantic Ocean 77309495550 C}KM7309495550 77309495550 PHYSICALASSETxVESSEL BALTIC PROSPERITY, Small, LAVINIA CORP, Caribbean Sea|Caribbean Tanker Zone 77309511101 C}KN7309511101 77309511101 PHYSICALASSETxVESSEL VASSILIO XVIII, Coastal, SANAHIL IO, West Atlantic Ocean (Americas)|Belem Tanker Zone 77309434563 C}KC7309434563 77309434563 PHYSICALASSETxVESSEL BALTIC, Aframax / LRII, ALPHA SHIPMANAGEMENT PTE LTD, South Pacific Ocean (NZ)|Melbourne Tanker Zone|Bass Strait Waypoint 77309466817 C}KA7309466817 77309466817 PHYSICALASSETxVESSEL MONT ACE, Handy, THOME SHIPPING PTE LTD, Louisiana|Gulf of Mexico|US Gulf Tanker Zone|United States 77309449300 C}KH7309449300 77309449300

In [12]:
// Use a wildcard (*) to search for references to 'Covid*'.
select = "DTSubjectName, DTSimpleType, DTSource, DTCharacteristics";

response = Search.Definition(Search.View.SearchAll).Query("Covid*")
                                                   .Select(select)
                                                   .GetData();
Console.WriteLine($"Matched a total of {response.Data.Total} documents");
response.Data.Hits

Matched a total of 7106 documents


DTSubjectName DTSimpleType DTSource DTCharacteristics Covidh Technologies Ltd Public Company Covidien Ventures Private Company Coronavirus (COVID-19), Total cases, Volume Reuters Polls Reuters Daily Coronavirus (COVID-19), Total cases, Volume Reuters Polls Reuters Daily Coronavirus (COVID-19), Total cases, Volume Reuters Polls Reuters Daily Coronavirus (COVID-19), Total cases, Volume Reuters Polls Reuters Daily Middle East and North Africa, Coronavirus (COVID-19), Total cases, Volume Reuters Polls Reuters Daily Coronavirus (COVID-19), Total cases, Volume Reuters Polls Reuters Daily Coronavirus (COVID-19), New cases, Volume Reuters Polls Reuters Daily Coronavirus (COVID-19), Active cases, Volume Reuters Polls Reuters Daily

In [13]:
// List the Vodafone bonds maturing in 2025 with a coupon rate of 5%
// We can achieve this specific query using a special feature called 'tcm' (ticker coupon maturity).
select = "CouponCurrency, CouponTypeDescription, IssuerLegalName, CouponRate, MaturityDate, ISIN";

response = Search.Definition(Search.View.SearchAll).Query("vod 5% 2025")
                                                   .Features("tcm")
                                                   .Select(select)
                                                   .GetData();
response.Data.Hits

USD,Fixed:Plain Vanilla Fixed Coupon,Vodafone Hessen GmbH & Co. KG,5,15/01/2025 12:00:00 AM,USD85455AD13
USD,Fixed:Plain Vanilla Fixed Coupon,Vodafone Hessen GmbH & Co. KG,5,15/01/2025 12:00:00 AM,US913364AD74
